# CHPATER 7. 함수 데커레이터와 클로저

**함수 데커레이터**는 소스코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다. 강력한 기능이지만, 데커레이터를 자유자재로 사용하려면 먼저 클로저를 알아야 한다. 데커레이터에서 사용하는 것 외에도, 클로저는 콜백을 이용한 효율적인 비동기 프로그래밍과 필요에 따라 함수형 스타일로 코딩하는 데에도 필수적이다.  
    
  
**배우는 것**
* 파이썬이 데커레이터 구문을 평가하는 방식
* 변수가 지역 변수인지 파이썬이 판단하는 방식
* 클로저의 존재 이유와 작동 방식
* `nonlocal`로 해결할 수 있는 문제
  
**데커레이터 주제**
* 잘 작동하는 데커레이터 구현하기
* 표준 라이브러리에서 제공하는 재미있는 데커레이터들
* 매개변수화된 데커레이터 구현하기

## 7.1 데커레이터 기본 지식
데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

In [1]:
def decorate(func):
    def inner():
        print('runing inner()')
    return inner

@decorate
def target():
    print('running target()')
    
target = decorate(target)

In [2]:
target
# target 객체는 target() 함수를 가리키는 것이 아니라 decorate(target)이 반환한 함수를 가리킨다.

<function __main__.decorate.<locals>.inner()>

In [3]:
# 예제 7-1 일반적으로 데커레이터는 함수를 다른 함수로 대체한다.
def deco(func):
    def inner():
        print('running inner()')
    return inner    # deco()는 inner() 함수 객체를 반환한다.

@deco    # target()을 deco로 데커레이트했다.
def target():
    print('running target()')

In [4]:
target()

running inner()


데커레이트된 `target()`을 호출하면 실제로는 `inner()`를 수행한다.

In [5]:
target

<function __main__.deco.<locals>.inner()>

`target`은 실제로 `inner()`를 가리키고 있음을 알 수 있다.  

데커레이터는 다른 함수를 인수로 전달해서 호출하는 일반적인 콜러블과 동일하다. 하지만 런타임에 프로그램 행위를 변경하는 **메타프로그래밍**을 할 때 데커레이터가 상당히 편리하다.  

*메타프로그래밍* : 런타임에 수행해야 할 작업의 일부를 컴파일 타임 동안 수행하는 프로그래밍.
  
파이썬에서는 `type()` 메타클래스를 이용.

### 요약
* 데커레이터는 데커레이트된 함수를 다른 함수로 대체하는 능력이 있다.
* 데커레이터는 모듈이 로딩될 때 바로 실행된다.

## 7.2 파이썬이 데커레이터를 실행하는 시점
데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후에 실행된다는 것이다. 이는 일반적으로 파이썬이 모듈을 로딩하는 시점, 즉 **임포트 타임**에 실행된다.

In [6]:
# 예제 7-2 registration.py 모듈
registry = []    # registry 배열은 @register로 데커레이트된 함수들에 대한 참조를 담는다.

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()

running register(<function f1 at 0x000001C8BFF81168>)
running register(<function f2 at 0x000001C8BFF81438>)
running main()
registry -> [<function f1 at 0x000001C8BFF81168>, <function f2 at 0x000001C8BFF81438>]
running f1()
running f2()
running f3()


`register()`는 `f1()`, `f2()`를 정의할 때 먼저 실행이 되고, 데커레이트된 함수를 인수로 받는다. (모듈이 로딩되는 시점)

In [7]:
import registration

running register(<function f1 at 0x000001C8BFF81AF8>)
running register(<function f2 at 0x000001C8BFF841F8>)


In [8]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

함수 데커레이터는 모듈이 임포트되자마자 실행이 되지만, 데커레이트된 함수는 명시적으로 호출될 때만 실행됨을 알 수 있다.  
파이썬 개발자가 **임포트 타임(import time)** 이라고 부르는 것과 **런타임(runtime)** 이라고 부르는 것의 차이를 보여준다.

[예제 7-2]와 실제 코드에 흔히 사용되는 방식과의 차이점
* 데커레이터 함수가 데커레이트되는 함수와 같은 모듈에 정의되어 있다. 일반적으로 실제 코드에서는 데커레이트 정의 모듈 / 데커레이터 적용 모듈을 분리해서 구현.
* `register()` 데커레이터가 인수로 전달된 함수와 동일한 함수를 반환. 실제 코드에서는 대부분의 데커레이터는 내부 함수를 정의해서 반환.
  
*데커레이터가 인수 함수를 반환하는 것이 꼭 쓸모없는 것은 아니다!*  
URL 패턴을 HTTP 응답 생성 함수에 매핑하는 레지스트리 등 함수를 어떤 중앙의 레지스트리에 추가하기 위해 Django 등에서 데커레이트를 사용하기도 한다.

## 7.3 데커레이터로 개선한 전략 패턴
6장에서 나왔던 프로모션 할인 코드를 개선해보자.  
아래 [예제 6-6]에서 `best_promo`는 `promos` 리스트에 함수명을 반복해서 사용하는데 만약, 새로운 프로모션 전략 함수를 `promos`에 깜빡하고 추가안했다면, 버그가 생길 수 있다.

In [9]:
# 예제 6-6 함수 리스트를 반복해서 최대 할인액을 찾아내는 best_promo() 함수
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

NameError: name 'fidelity_promo' is not defined

In [10]:
# 예제 7-3 promotion 데커레이터로 채운 promos 리스트
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

대부분의 데커레이터는 데커레이트된 함수를 변경한다. 즉, 내부 함수를 정의하고 그것을 반환하여 데커레이트된 함수를 대체한다. 내부 함수를 사용하는 코드는 제대로 작동하기 위해 거의 항상 클로저에 의존한다. 다음 장에선, 클로저를 이해하기 위해 먼저 파이썬에서 변수 범위의 작동 방식에 대해 살펴본다.

## 7.4 변수 범위 규칙

In [11]:
# 예제 7-4 지역 및 전역 변수를 읽는 함수
def f1(a):
    print(a)
    print(b)

In [12]:
f1(3)

3


NameError: name 'b' is not defined

전역 변수 `b`에 값을 할당하고 `f1()`을 호출하면 제대로 작동한다.

In [13]:
b = 6
f1(3)

3
6


In [14]:
# 예제 7-5 함수 본체 안에서 값을 할당하기 때문에 지역 변수가 되는 b
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [15]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

글쓴이가 놀란 이유 : 전역 변수 `b`가 있고 `print(b)` 다음에 지역 변수 `b`에 할당하는 문이 나오므로 전역 변수의 값인 6이 출력될 것이라고 생각했기 때문.  

사실은 파이썬이 함수 본체를 컴파일할 때 `b`가 함수 안에서 할당되므로 `b`를 지역 변수로 판단한다.  
`global` 키워드를 이용해서 문제를 해결하자.

In [16]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [17]:
f3(3)

3
6


In [18]:
b

9

In [19]:
b = 30
b

30

`f1()`와 `f2()`의 바이트코드를 비교해보자.

In [20]:
from dis import dis
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [21]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


`f1()` 바이트 코드의 10번째 줄과 `f2()` 바이트 코드의 10번째 줄을 비교해보면, `f1()`은 `b`를 전역 변수로 간주하고, `f2()`는 지역 변수로 간주함을 알 수 있다.

## 7.5 클로저
클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonglobal) 변수를 포함한 확장 범위를 가진 함수다.   
-> 어떤 함수를 함수 자신이 가지고 있는 환경과 함께 저장한 레코드.   
함수가 익명 함수인지 여부는 중요하지 않고 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다.

In [22]:
# 예제 7-8 average_oo.py : 이동 평균을 계산하는 클래스
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [23]:
avg = Averager()

In [24]:
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

In [27]:
avg.series

[10, 11, 12]

In [28]:
# 예제 7-9 average.py : 이동 평균을 계산하는 고위 함수
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

호출되면 `make_averager()`는 `average()` 함수 객체를 반환한다. `average()` 함수는 호출될 때마다 받은 인수를 `series` 리스트에 추가하고 [예제 7-10]과 같이 현재까지의 평균을 계산해서 출력한다.

In [29]:
# 예제 7-10 [예제 7-9] 테스트
avg = make_averager()

In [30]:
avg(10)

10.0

In [31]:
avg(11)

10.5

In [32]:
avg(12)

11.0

`make_average()` 함수 본체 안에서 `series = []`로 초기화하고 있기 때문에 `series`는 이 함수의 지역 변수다.  
그렇지만 `avg(10)`을 호출할 때, `make_average()` 함수는 이미 반환했으므로 지역 범위도 이미 사라진 후다.   

`average` 안에 들어 있는 `series`는 자유 변수(free variable)다.  
자유 변수(free variable) : 코드블럭 안에서 사용은 되었지만, 그 코드블럭 안에서 정의되지 않은 변수.

![7-2.png](./7-2.png)

In [33]:
# 예제 7-11 [예제 7-9]의 make_averager()로 생성한 함수 조사하기
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [35]:
avg.__closure__

(<cell at 0x000001C8BFF69708: list object at 0x000001C8C131FA48>,)

In [36]:
avg.__closure__[0].cell_contents

[10, 11, 12]

클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수다.  
따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.  
함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우뿐이라는 점에 주의하자. 

## 7.6 nonlocal 선언
앞에서 구현한 `make_averager()`는 효율적이지 않다. 왜냐하면 `average()`가 호출될 때마다 `sum`을 다시 계산했기 때문.  
합계와 항목 수를 저장한 후 이 두 개의 숫자를 이용해서 평균을 구하면 훨씬 더 효율적으로 구현할 수 있다.

In [37]:
# 예제 7-13 전체 이력을 유지하지 않고 이동 평균을 계산하는 잘못된 고위 함수
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [38]:
avg = make_averager()

In [39]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`count`에 `count + 1`이라는 할당을 하는 과정에서, `count`를 지역 변수로 만들기 때문에 문제가 생긴다. `count`가 더 이상 자유 변수가 아니므로 클로저에 저장되지 않는다. `total`도 마찬가지. 

이 문제는 `nonlocal` 선언으로 해결할 수 있다. 변수를 `nonlocal`로 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유 변수 임을 나타내준다.  

`nonlocal`은 사용된 함수 바로 한 단계 바깥 쪽에 위치한 변수와 바인딩한다는 점에서 `global`과 차이가 있다.

In [40]:
# 예제 7-14 전체 이력을 유지하지 않고 이동 평균 계산하기(nonlocal로 수정)
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [41]:
avg = make_averager()

In [42]:
avg(10)

10.0

## 7.7 간단한 데커레이터 구현하기

In [43]:
# 예제 7-15 함수의 실행 시간을 출력하는 간단한 데커레이터
# clockdeco.py
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter() # 전체적인 시간을 모두 측정
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:.8f}s] {name}({arg_str}) -> {result}')
        return result
    return clocked

In [44]:
# 예제 7-16 clock 데커레이터 사용하기
# clockdeco_demo.py
import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12273630s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000110s] factorial(1) -> 1
[0.00060150s] factorial(2) -> 2
[0.00131100s] factorial(3) -> 6
[0.00183400s] factorial(4) -> 24
[0.00232720s] factorial(5) -> 120
[0.00279530s] factorial(6) -> 720
6! = 720


### 7.7.1 작동 과정

In [45]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

위 코드는 실제로 다음 코드로 실행된다.

In [46]:
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

factorial = clock(factorial)

앞의 두 예제에서 `clock()`은 `factorial()` 함수를 `func` 인수로 받는다. 그 후 `clock()` 함수를 만들어서 반환하는데, 파이썬 인터프리터가 내부적으로 `clocked()`를 `factorial`에 할당했다. 실제로 `clockdeco_demo` 모듈을 임포트해서 `factorial`의 `__name__`속성을 조사해보면 아래와 같다.

In [47]:
import clockdeco_demo
clockdeco_demo.factorial.__name__

'clocked'

`factorial`은 실제로 `clocked()` 함수를 참조한다. 이제부터 `factorial(n)`을 호출하면 `clocked(n)`이 실행된다. 본질적으로 `clocked()` 함수는 다음과 같은 연산을 수행한다.
1. 초기 시작 `t0`를 기록한다.
2. 원래의 `factorial()` 함수를 호출하고 결과를 저장한다.
3. 흘러간 시간을 계산한다.
4. 수집한 데이터를 포맷하고 출력한다.
5. 2번째 단계에서 저장한 결과를 반환한다.

[예제 7-15]에서 구현한 `clock()` 데커레이터의 단점  
* 키워드 인수를 지원하지 않음
* 데커레이트된 함수의 `__name__`과 `__doc__` 속성을 가린다.

In [48]:
# 예제 7-17 개선된 clock 데커레이터
# clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func) # 표준 라이브러리에서 제공하는 데커레이터 중 하나.
    def clocked(*args, **kwargs): # keyword 인수를 지원.
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:.8f}] {name}({arg_str}) -> {result}')
        return result
    return clocked

In [49]:
import clockdeco_demo2

In [50]:
clockdeco_demo2.factorial.__name__

'factorial'

## 7.8 표준 라이브러리에서 제공하는 데커레이터
파이썬은 메서드의 데커레이트를 위해 `property()`, `classsmethod()`, `staticmethod()` 등 3개의 내장 함수를 제공한다.  
`functools.wraps()`은 데커레이터를 만들기 위한 헬퍼. 

### 7.8.1 functools.lru_cache()를 이용한 메모이제이션

In [52]:
# 예제 7-18 fibo_demo.py : 피보나치 수열에서 n번째 숫자를 아주 값비싸게 계산하는 방식
from clockdeco import clock

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000060s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00008220s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00002410s] fibonacci(2) -> 1
[0.00034850s] fibonacci(3) -> 2
[0.00052490s] fibonacci(4) -> 3
[0.00000050s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00002780s] fibonacci(2) -> 1
[0.00006300s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00002600s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000030s] fibonacci(1) -> 1
[0.00003020s] fibonacci(2) -> 1
[0.00005710s] fibonacci(3) -> 2
[0.00010780s] fibonacci(4) -> 3
[0.00019670s] fibonacci(5) -> 5
[0.00081990s] fibonacci(6) -> 8
8


같은 계산이 여러 번 호출되는 등 계산 낭비가 많다.

In [53]:
# 예제 7-19 캐시를 이용한 더 빠른 구현
import functools 

from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

[0.00000060s] fibonacci(0) -> 0
[0.00000080s] fibonacci(1) -> 1
[0.00031840s] fibonacci(2) -> 1
[0.00000080s] fibonacci(3) -> 2
[0.00036230s] fibonacci(4) -> 3
[0.00000060s] fibonacci(5) -> 5
[0.00039590s] fibonacci(6) -> 8
8


실행 시간이 절반으로 줄었고, 각 n에 대해 함수가 딱 한번만 호출된 것을 볼 수 있다.

`lru_cache()`는 웹에서 정보를 가져와야하는 애플리케이션에서도 진가를 발휘한다.

In [54]:
functools.lru_cache(maxsize=128, typed=False)

<function functools.lru_cache.<locals>.decorating_function(user_function)>

`maxsize`인수는 얼마나 많은 호출을 저장할지 결정. 캐시가 가득차면 가장 오래된 결과를 버린다.  
최적의 성능을 내기 위해선 `maxsize`는 2의 제곱이 되어야 한다. -> ???  
`typed` 인수는 `True`로 설정되는 경우 인수의 자료형이 다르면 결과를 따로 저장한다.  
  
`lru_cache()`가 결과를 저장하기 위해 딕셔너리를 사용하고, 호출할 때 사용한 위치 인수와 키워드 인수를 키로 사용하기 때문에, 데커레이트된 함수가 받는 인수는 모두 **해시 가능**해야 한다.

### 7.8.2 단일 디스패치를 이용한 범용 함수

In [70]:
# 파이썬 객체의 자료형마다 HTML 코드를 생성하고자 함.
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

위 함수를 아래의 조건을 고려하도록 확장하고자 한다.
* `str` : 개행 문자를 '`<br>\n`'으로 대체하고 `<pre>`대신 `<p>` 태그를 사용한다.
* `int` : 숫자를 10진수와 16진수로 보여준다.
* `list` : 각 항목을 자료형에 따라 포맷한 HTML 리스트를 출력한다.
  
파이썬은 메서나 함수의 오버로딩을 지원하지 않으므로, 서로 다르게 처리하고자 하는 자료형별로 서로 다른 시그너처를 가진 `htmlize()`를 만들 수 없다.

In [71]:
# 예제 7-20 다른 객체형에 맞춰진 HTML을 생성하는 htmlize()
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [72]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [73]:
htmlize('Heimlich & Co.\n- a game')

'<pre>&#x27;Heimlich &amp; Co.\\n- a game&#x27;</pre>'

In [74]:
htmlize(42)

'<pre>42</pre>'

In [75]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


일반 함수를 `@singledispatch`로 데커레이트하면, 이 함수는 **범용 함수**(generic function)이 된다. 즉, 일련의 함수가 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.

In [76]:
# 예제 7-21 여러 함수를 범용 함수로 묶는 커스텀 htmlize.register()를 생성하는 singledispatch
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch    # @singledispatch()는 객체형을 다룰 기반 함수를 표시한다.
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)    # number.Integral은 int의 가상 수퍼클래스
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

* 각각의 특화된 함수는 `@<기반_함수>.register(<객체형>)`으로 데커레이트된다.  
* 동일한 함수로 여러 자료형을 지원하기 위해 `register` 데커레이터를 여러 개 쌓아올릴 수 있다.

In [77]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [78]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [79]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [80]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [81]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


가능하면 `int`나 `list`와 같은 구상 클래스보다 `numbers.Integral`이나 `abc.MutableSequence`와 같은 추상 베이스 클래스를 처리하도록 특화된 함수를 등록하는 것이 좋다. 추상 베이스 클래스로 등록하면 호환되는 자료형을 폭넓게 지원할 수 있다.(11장에서 다룬다고 함.)

## 7.9 누적된 데커레이터
하나의 함수 `f()`에 두 데커레이터 `@d1`과 `@d2`를 차례대로 적용하면, 결과는 `f = d1(d2(f))`와 동일하다.

In [82]:
@d1
@d2
def f():
    print('f')

NameError: name 'd1' is not defined

위 코드는 다음 코드와 동일하다.

In [83]:
def f():
    print('f')
    
f = d1(d2(f))

NameError: name 'd1' is not defined

## 7.10 매개변수화된 데커레이터
소스코드에서 데커레이터를 파싱할 때 파이썬은 데커레이트된 함수를 가져와서 데커레이터 함수의 첫 번째 인수로 넘겨준다.  
그러면 어떻게 다른 인수를 받는 데커레이터를 만들 수 있을까?  
-> 인수를 받아 데커레이터를 반환하는 데커레이터 팩토리를 만들고 나서, 데커레이트될 함수에 데커레이터 팩토리를 적용하면 된다.

In [84]:
# 예제 7-22 [예제 7-2]의 registration.py 축약 버전
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x000001C8BFF811F8>)
running main()
registry -> [<function f1 at 0x000001C8BFF811F8>]
running f1()


### 7.10.1 매개변수화된 등록 데커레이터
`register()`가 등록하는 함수를 활성화 혹은 비활성화하기 쉽게 만들기 위해, 선택적인 인수 `active`를 받도록 만들어보자. `active`가 `False`면 데커레이트된 함수를 등록 해제한다. 새로 만든 `register()` 함수는 개념적으로는 데커레이터가 아니라 데커레이터 팩토리다. 호출되면 대상 함수에 적용할 실제 데커레이터를 반환하기 때문이다.

In [85]:
# 예제 7-23 매개변수를 받기 위해 함수로 호출되어야 하는 새로운 register() 데커레이터 : registration_param.py

registry = set()

def register(active=True):
    def decorate(func):
        print(f'running register(active={active})->decorate{func}')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

running register(active=False)->decorate<function f1 at 0x000001C8C13FB168>
running register(active=True)->decorate<function f2 at 0x000001C8C13FB288>


`register()`가 `decorate()`를 반환하고, 데커레이트될 함수에 `decorate()`가 적용된다.

In [88]:
import registration_param

running register(active=False)->decorate<function f1 at 0x000001C8C1403318>
running register(active=True)->decorate<function f2 at 0x000001C8C1403558>


In [89]:
registration_param.registry

{<function registration_param.f2()>}

`f2()`함수만 `registry`에 남아있다. `register()`데커레이터 팩토리에 `active=False` 인수를 전달했으므로 `f1()`에 적용된 `decorate()`는 `registry`에 `f1()` 함수를 추가하지 않았다.

In [90]:
# 예제 7-24 [예제 7-23]에 나열된 registration_param 모듈 사용하기
from registration_param import *

임포트 타임에만 데코레이터가 실행된다.

In [91]:
registry

{<function registration_param.f2()>}

In [92]:
register()(f3)

running register(active=True)->decorate<function f3 at 0x000001C8C1403288>


<function registration_param.f3()>

커머셜 앳(`@`) 구문을 사용하는 대신 `register()`를 일반 함수로 사용하려면 괄호를 사용한다.  
`register()` 표현식은 `decorate()`를 반환하고, 이 데커레이터가 `f3()`에 적용된다.

In [93]:
registry

{<function registration_param.f2()>, <function registration_param.f3()>}

In [94]:
register(active=False)(f2)

running register(active=False)->decorate<function f2 at 0x000001C8C1403558>


<function registration_param.f2()>

In [95]:
registry

{<function registration_param.f3()>}

### 7.10.2 매개변수화된 clock 데커레이터
아래 예제는 사용자가 포맷 문자열을 전달해서 데커레이트된 함수가 출력할 문자열을 설정한다.

In [96]:
# 코드를 간단히 하기 위해 [예제 7-25]는 @functools.wraps()를 이용해서 
# 함수 계층을 한 단계 더 추가하는 개선된 [예제 7-17] 대신, [예제 7-15]에서 구현한
# 초기 clock() 예제에 기반해서 구현했다.

# 예제 7-25 clockdeco_param.py 모듈 : 매개변수화된 clock() 데커레이터

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):                 # 매개변수화된 데커레이터 팩토리
    def decorate(func):                     # 실제 데커레이터
        def clocked(*_args):                # 데커레이트된 함수를 래핑
            t0 = time.time()
            _result = func(*_args)          # 데커레이트된 함수의 결과를 저장
            elapsed = time.time() - t0 
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
#             print('args :', args)         # args : 0.123
            result = repr(_result)
#             print([local for local in locals()])
#             ['_args', 't0', '_result', 'elapsed', 'name', 'args', 'result', 'fmt', 'func']
            print(fmt.format(**locals()))   # fmt가 clocked()의 지역 변수를 모두 참조할 수 있게 해준다.
            return _result
        return clocked
    return decorate

if __name__ == '__main__':
    
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze(.123)

[0.12367082s] snooze(0.123) -> None
[0.12366915s] snooze(0.123) -> None
[0.12366939s] snooze(0.123) -> None


In [1]:
# 예제 7-26 clockdeco_param_demo1.py

import time
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12366914749145508s
snooze: 0.1237344741821289s
snooze: 0.12360620498657227s


In [2]:
# 예제 7-27 clockdeco_param_demo2.py
import time
from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.124s


## 7.11 요약
* 매개변수화된 데커레이터는 거의 항상 최소 두 단계의 내포된 함수를 가지고 있다. 
* 고급 기법을 지원하는 데커레이터를 구현하기 위해 `@functools.wraps`를 사용하는 경우 세 단계 이상 내포
* 표준 라이브러리의 `functools` 모듈에서 제공하는 `@lru_cache()`와 `@singledispatch`를 사용했다.
* **임포트 타임**과 **런타임**의 차이를 알았다.